# 11. 뉴스 요약봇 만들기
## 11-2. 텍스트 요약(Text Summarization)
- 긴 길이의 문서(Document) 원문을 핵심 주제만으로 구성된 짧은 요약(Summary) 문장들로 변환하는 것

```text
ex) 뉴스기사 (큰 텍스트) --> 뉴스 제목 (작은 텍스트)

중요한점: 요약 전후에 정보 손실 발생이 최소화되어야 한다.
```

요약 문장을 만드는 방법?</br>
1. 추출적 요약(Extractive Summarization)
    - 원문에서 **문장들을 추출**해서 요약하는 방식
    - 딥러닝 보다는 주로 전통적인 머신러닝 방식에서 사용
        - 텍스트 랭크(Text Rank)
            - 각 문장에 중요도(랭크)를 설정하여 중요한 문장 몇 가지를 추출하는 방식
        - 대표적인 사용처
            - 네이버 뉴스 서비스에 있는 `요약봇`
                - 요약문에 사용된 문장 3개가 원문에 그대로 있는것이 특징
2. 추상적 요약(Abstractive Summarization)
    - 원문으로부터 내용이 요약된 **새로운 문장을 생성**
        - 새로운 문장은 원문에 원래 없던 문장일 수도 있다.
    - 자연어 처리 분야 중 하나를 사용
        - 자연어 생성(Natural Language Generation, NLG) 기법
    - 원문을 구성하는 문장 중 어느 것이 요약문에 들어갈 핵심문장인지를 판별한다는 점
        - 문장 분류(Text Classification) 문제로 볼 수도 있다.
    - 기본적인 신경망 접근법
        - RNN
            - 먼 과거의 정보를 현재에 전달하기 어려운 `장기 의존성(Long Term Dependencies)` 문제가 있다.
    - 구글의 텍스트 요약 모델의 문장 요약법
        - 역문서빈도(IDF) 지표
            - 문서 안에서 중요해 보이는 부분을 추출하여 요약문에 담는 방식
            - 이런 발췌 방식은 어색하거나 문법적으로 이상한 결과물을 만들 수 있다. (ex: `앨리스와 밥은 동물원에 가다. 조류 한 무리를 봤다.`)
        - 추상적 요약(Abstractive Summarization)
            - ex: `앨리스와 밥은 동물원에 갔고 짐승과 새를 봤다.`
    - seq2seq
        - 짧은 문장, 요약문을 생성하는 구글의 모델
        - 딥러닝을 통해 `end-to-end`로 설계

## 11-3. 인공 신경망으로 텍스트 요약 훈련시키기
**seq2seq 개요**</br>
![Summary of seq2seq](../images/23.png)</br>
[Link](https://medium.com/dl-for-product-and-service/abstractive-text-summary-with-reinforcement-learning-ab2458ab29d5)</br>

원문을 첫 번째 RNN인 encoder로 입력하면 encoder는 이를 하나의 고정된 벡터로 변환</br>
이 벡터를 문맥 정보를 가진 벡터; `컨택스트 벡터(Context vector)`라고 한다.</br>

두 번째 RNN인 decoder는 이 컨택스트 벡터를 전달받아 한 단어씩 생성하면서 요약 문장을 완성</nr>

**LSTM과 컨텍스트 벡터**</br>
seq2seq를 구현할 때, 인코더/디코더로 vanilla RNN이 아니고 LSTM을 사용할 것임.</br>
![RNN과 LSTM](../images/24.png)</br>
[Link](https://colah.github.io/posts/2015-08-Understanding-LSTMs)</br>

LSTM은 다음 time step의 셀에 `hidden state`와 `cell state`도 함께 전달한다.</br>
(Vanilla RNN과 다른점)</br>

**시작 토큰과 종료 토큰**</br>
![시작 토큰 SOS와 종료 토큰 EOS는 각각 start of a sequence와 end of a sequnce를 나타낸다.](../images/25.png)</br>
[Link](https://arxiv.org/pdf/1812.02303.pdf)</br>

seq2seq 구조에서 디코더는 시작 토큰 SOS가 입력되면, 각 시점마다 단어를 생성하고 이 과정을 종료 토큰 EOS를 예측하는 순간 멈추지 않는다.</br>
즉 시작 토큰과 종료 토큰을 넣어줄 때 (`a.k.a 전처리`) 어디서 멈춰야 할지를 알려줘야 한다.</br>

**어텐션 메커니즘을 통한 새로운 컨텍스트 벡터 사용하기**</br>
![기존의 컨텍스트 벡터보다 인코더의 정보를 적극적으로 활용하여 성능을 끌어올리는 어텐션 메커니즘](../images/26.png)</br>
[Link](https://arxiv.org/pdf/1812.02303.pdf)</br>

기존의 seq2seqe는 **인코더의 마지막 time step의 hidden state**를 컨택스트 벡터로 사용하였다.</br>
 - RNN 계열의 인공 신경망 (RNN, LSTM, GRU)의 한계로 인해 이 컨택스트 정보에는 이미 입력 시퀀스의 많은 정보가 손실이 된 상태다.

**어텐션 메커니즘(Attention Mechanism)**은 이와 달리, **인코더의 모든 step의 hidden state의 정보**가 컨텍스트 벡터에 전부 반영되도록 한다.</br>
 - hidden state는 모든 인코더에 동일한 비중으로 반영되지 않는다.
 - 디코더의 현재 time step의 예측 + 인코더의 각 step이 얼마나 영향을 미치는지에 따른 가중합으로 계산되는 방식

주의점</br>
- 컨텍스트 백터를 구성하기 위한 인코더의 hidden state의 가중치 값은 `디코더의 현재 스텝에 따라 계속 달라진다.`
    - 디코더의 현재 문장 생성 부위가 주어부인지 술어부인지 목적어인지 등에 따라 인코더가 입력 데이터를 해석한 컨텍스트 벡터가 다른 값이 된다.

## 11-4. 데이터 준비하기
작업환경 구성(노드 기준)</br>
```shell
$ mkdir -p ~/aiffel/news_summarization/data
```

사용할 데이터셋은 Kaggle에서 제공된 `아마존 리뷰 데이터셋`</br>
[Link: Amazon Fine Food Revies](https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews)</br>

클라우드 데이터셋 이용을 위한 추가 작업
```shell
$ ln -s ~/data/*.csv ~/aiffel/news_summarization/data
```

이번 실습에서는 NLTK의 불용어(stopwords)를 사용함
 - NLTK: Natural Language Toolkit
 - 영어 기호, 통계, 자연어 처리를 위한 라이브러리
 - 기본적으로 100여 개의 불용어를 가지고 있음

In [ ]:
import nltk
nltk.download('stopwords')

import numpy as np
import pandas as np
import os
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

print('=3')

```shell
[nltk_data] Downloading package stopwords to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip


=3
```

링크에서 다운받은 데이터(`Reviews.csv`)는 총 568,454개의 샘플을 가지고 있다.</br>
테스트에서는 10만개만을 사용한다.

In [ ]:
data = pd.read_csv(os.getenv("HOME")+"/aiffel/news_summarization/data/Reviews.csv", nrows=100000)
print(f"전체 샘플수: {len(data)}")

```shell
전체 샘플수 : 100000
```

전체 결과 중 5개만 출력</br>

```python
data.head()
```

![Dataframe head](../images/27.png)</br>

전체 데이터 중 Summary, Text 열들만 별도로 저장 후 출력해보기

In [ ]:
# input [[YOUR CODE]] aside `data`
data = data[['Text', 'Summary']]
data.head()

# 랜덤한 15개 샘플 출력
data.sample(15)

![15 random samples, Text & Summary cols](../images/28.png)</br>

인공 신경망을 통해 `Text`를 입력받으면 `Summary`를 예측하도록 인공 신경망을 훈련시킬 것이다.

## 11-5. 데이터 전처리하기
### (1) 데이터 정리하기

**중복 샘플과 NULL 값이 존재하는 샘플 제거**</br>
데이터의 중복 샘플 유무 확인

In [ ]:
print(f"Text 열에서 중복을 배제한 유일한 샘플의 수: {data['Text'].nunique()}")
print(f"Summary 열에서 중복을 배제한 유일한 샘플의 수: {data['Summary'].nunique()}")

```shell
Text 열에서 중복을 배제한 유일한 샘플의 수 : 88426
Summary 열에서 중복을 배제한 유일한 샘플의 수 : 72348
```

Text 자체가 중복일 필요는 없으므로 데이터프레임의 `drop_duplicates()`사용하여 중복 샘플 제거

In [ ]:
# inplace = True 가 설정되면 DataFrame 타입 값을 return 하지 않고 data 내부를 직접적으로 변경함
data.drop_duplicates(subset = ['Text'], inplace=True)
print(f"전체 샘플 수 : {len(data)}")

```shell
전체 샘플수 : 88426
```

`drop_duplicates()`가 중복된 Null들을 지워주지만, null값이 남아있을 수 있으므로, 확인해보자</br>
확인하는 방법은 `.isnul().sum()`을 사용하자.

In [ ]:
print(data.isnull().sum())

```shell
Text       0
Summary    1
dtype: int64
```

Summary에 1개의 null 값이 있음을 확인.</br>
데이터프레임에서 Null을 제거할 때는 `dropna()` 함수 사용하기

In [ ]:
data.dropna(axis=0, inplace=True)
print(f"전체 샘플 수 : {len(data)}")

```shell
전체 샘플수 : 88425
```

**텍스트 정규화와 불용어 제거**</br>
텍스트 정규화
 - 같은 의미를 가진 단어지만 다르게 사용되는 경우에 같은 단어적 표현으로 통일시키는 방법
    - ex: `It'll`과 `It will`, `Mustn't`와 `Must not`은 서로 같은 표현임

텍스트 정규화를 위한 사전 생성 코드를 아래에서 보자</br>
- [정규화 사전 출처](https://stackoverflow.com/questions/19790188/expanding-english-lanuage-contractions-in-python)</br>

In [ ]:
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

print("정규화 사전의 수: ", len(contractions))

```shell
정규화 사전의 수 : 120
```

NLTK에서 제공하는 불용어 리스트 참고하기

In [ ]:
print(f"불용여 개수: {len(stopwords.words('english'))}")
print(stopwords.words('english'))

```shell
불용어 개수 : 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
```

In [ ]:
# 데이터 전처리 함수
def preprocess_sentence(sentence, remove_stopwords=True):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (...) 제거 Ex) my husband (and myself!) for => my husband for
    sentence = re.sub('"','', sentence) # 쌍따옴표 " 제거
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")]) # 약어 정규화
    sentence = re.sub(r"'s\b","", sentence) # 소유격 제거. Ex) roland's -> roland
    sentence = re.sub("[^a-zA-Z]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub('[m]{2,}', 'mm', sentence) # m이 3개 이상이면 2개로 변경. Ex) ummmmmmm yeah -> umm yeah
    
    # 불용어 제거 (Text)
    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stopwords.words('english') if len(word) > 1)
    # 불용어 미제거 (Summary)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    return tokens
print('=3')

```shell
=3
```

전처리 전, 후의 결과를 확인하기 위해 임의의 text와 summary를 만들어 함수 호출

In [ ]:
temp_text = 'Everything I bought was great, infact I ordered twice and the third ordered was<br />for my mother and father.'
temp_summary = 'Great way to start (or finish) the day!!!'

print("text: ", preprocess_sentence(temp_text))
print("summary:", preprocess_sentence(temp_summary, False))  # 불용어를 제거하지 않습니다.

```shell
text: everything bought great infact ordered twice third ordered wasfor mother father
summary: great way to start the day
```

특징
- 모든 알파벳이 소문자로 변환
- HTML 태그 제거 (`<br />` 등)
- 괄호 단어 (`(or finish)`) 제거
- 특수 문자 제거

훈련 데이터 전체에 대해서 전처리 수행하기
- Text의 경우 불용어 제거
- Summary의 경우 불용어를 제거하지 않음
- 전처리 이후 결과 확인 시 상위 5개의 줄 출력

In [ ]:
# Q. 위의 내용을 참고해서 훈련 데이터 전체의 Text 컬럼의 데이터를 전처리하는 코드를 작성하세요. (반복문 사용)

# 전체 Text 데이터에 대한 전처리 : 10분 이상 시간이 걸릴 수 있습니다.
clean_text = []

# [[YOUR CODE]]
for sentence in data['Text']:
    clean_text.append(preprocess_sentence(sentence, remove_stopwords=True))


# 전처리 후 출력
print("Text 전처리 후 결과: ", clean_text[:5])

```shell
Text 전처리 후 결과:  ['bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better', 'product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo', 'confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story lewis lion witch wardrobe treat seduces edmund selling brother sisters witch', 'looking secret ingredient robitussin believe found got addition root beer extract ordered made cherry soda flavor medicinal', 'great taffy great price wide assortment yummy taffy delivery quick taffy lover deal']
```

In [ ]:
# Q. 위의 내용을 참고해서 훈련 데이터 전체의 Summart 컬럼의 데이터를 전처리하는 코드를 작성하세요. (반복문 사용)

# 전체 Text 데이터에 대한 전처리 : 10분 이상 시간이 걸릴 수 있습니다.
clean_summary = []

# [[YOUR CODE]]
for sentence in data['Text']:
    clean_summary.append(preprocess_sentence(sentence, remove_stopwords=True))


# 전처리 후 출력
print("Text 전처리 후 결과: ", clean_summary[:5])

```shell
Summary 전처리 후 결과:  ['good quality dog food', 'not as advertised', 'delight says it all', 'cough medicine', 'great taffy']
```

텍스트 정제 이후 빈(empty) 샘플이 있는지 확인이 필요하다.</br>
빈 값이 있으면 모두 null 값을 가진 샘플로 대체하자.

In [ ]:
data['Text'] = clean_text
data['Summary'] = clean_summary

# 빈 값을 Null 값으로 변환
data.replace('', np.nan, inplace=True)
print('=3')

```shell
=3
```

이전과 같이 `.isnull().sum()`을 사용해서 Null 값이 생겼는지 확인하기.

In [ ]:
data.isnull().sum()

```shell
Text        0
Summary    70
dtype   int64
```

Summary 열에서 70개의 Null 값이 생겼다.
- 원래는 단어가 있었는데 정제 과정에서 모든 단어가 제거되어 빈 샘플이 70개 생겼다는 의미

이 샘플들을 모두 제거하자

In [ ]:
data.dropna(axis=0, inplace=True)
print(f"전체 샘플수: {len(data)}")

```shell
전체 샘플수: 88355
```

## 11-6. 데이터 전처리하기
### (2) 훈련데이터와 테스트데이터 나누기

<!-- 추가 정리 필요 -->